### Data Collection

We will use a package called `BeautifulSoup` to webscrap and collect the data from the web. The data will then be saved in a local `csv` file for further prprocessing.

We will scrap the reviews from [https://www.airlinequality.com], using `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os
import re
%matplotlib inline

In [2]:
# We are only interested in reviews for British Airways
base_url = "https://www.airlinequality.com/airline-reviews/british-airways/"
pages = 20
page_size = 100

reviews = []
stars = []
date = []
country = []

for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')

    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())

    # collect all stars
    for para in parsed_content.find_all("div", {"class": "rating-10"}):
        try:
            stars.append(para.span.get_text())
        except:
            stars.append(np.nan)

    # collect all dates
    for para in parsed_content.find_all("time"):
        date.append(para.get_text())

    # collect country
    for para in parsed_content.find_all("h3"):
        country.append(para.span.next_sibling.text.strip(" ()"))

    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews
Scraping page 11
   ---> 1100 total reviews
Scraping page 12
   ---> 1200 total reviews
Scraping page 13
   ---> 1300 total reviews
Scraping page 14
   ---> 1400 total reviews
Scraping page 15
   ---> 1500 total reviews
Scraping page 16
   ---> 1600 total reviews
Scraping page 17
   ---> 1700 total reviews
Scraping page 18
   ---> 1800 total reviews
Scraping page 19
   ---> 1900 total reviews
Scraping page 20
   ---> 2000 total reviews


In [3]:
# check the length of the lists
print(len(reviews))
print(len(stars))
print(len(date))
print(len(country))

2000
2020
2000
2000


In [4]:
# investigate the stars list
stars[:20]

['\n\t\t\t\t\t\t\t\t\t\t\t\t\t5',
 '1',
 '9',
 '2',
 '7',
 '7',
 '1',
 '4',
 '1',
 '2',
 '1',
 '5',
 '1',
 '1',
 '1',
 '6',
 '1',
 '9',
 '9',
 '3']

In [5]:
# remove first entry of stars
stars = stars[1:]

# check if all lists have the same length
print(len(reviews), len(stars), len(date), len(country))

# make length of stars equal to reviews
stars = stars[:len(reviews)]

# check again
print(len(reviews), len(stars), len(date), len(country))

2000 2019 2000 2000
2000 2000 2000 2000


In [6]:
# create list of (review, stars, date, country) tuples
data = list(zip(reviews, stars, date, country))

# create dataframe
df = pd.DataFrame(data, columns=["review", "stars", "date", "country"])

# convert date to datetime
df["date"] = pd.to_datetime(df["date"])

# convert stars to float
df["stars"] = df["stars"].astype(float)

# see first 5 rows
df.head()

,review,stars,date,country
0,✅ Trip Verified | I was flying to Warsaw for ...,1.0,2022-12-03,United States
1,✅ Trip Verified | Booked a BA holiday to Marr...,9.0,2022-11-30,United Kingdom
2,✅ Trip Verified | Extremely sub-par service. H...,2.0,2022-11-28,United States
3,✅ Trip Verified | I virtually gave up on Brit...,7.0,2022-11-26,United Kingdom
4,✅ Trip Verified | I was pleasantly surprised ...,7.0,2022-11-25,Canada


In [7]:
df.to_csv("data/reviews.csv")

We have collected about 2000 reviews by iterating through the paginated pages on the website. To increase the number of data collected, we just have to increase the page count. In the next step, we will do cleaning on the data.